# **Template OP on salish**

In [24]:
%matplotlib inline

import sys
import xarray as xr
import numpy as np
import os
import math
from datetime import datetime, timedelta
from parcels import FieldSet, Field, VectorField, ParticleSet, JITParticle, ErrorCode, ParcelsRandom

sys.path.append('/home/jvalenti/MOAD/analysis-jose/notebooks/parcels')
from Kernels_sink import DeleteParticle, Buoyancy, AdvectionRK4_3D, Stokes_drift
from OP_functions import *

In [25]:
# Define paths
local = 0 #Set to 0 when working on server
paths = path(local)

In [26]:
Dat=xr.open_dataset(get_WW3_path(datetime(2018, 12, 23)))
path_NEMO = make_prefix(datetime(2018, 12, 23), paths['NEMO'])
Dat0=xr.open_dataset(path_NEMO + '_grid_W.nc')
Dat=xr.open_dataset(path_NEMO + '_grid_T.nc')
batt=xr.open_dataset(paths['mask'],decode_times=False)
coord=xr.open_dataset(paths['coords'],decode_times=False)
WW3 = xr.open_dataset(get_WW3_path(datetime(2018, 12, 23)))

In [27]:
# clat,clon = p_unidist(coord.gphif[0,:,:],coord.glamf[0,:,:],batt.mbathy[0,:,:],10,10)
# with open("clat.txt", "w") as output0:
#     output0.write(str(clat))
# with open("clon.txt", "w") as output:
#     output.write(str(clon))

with open('clat2.txt') as f:
    clat = f.read()
    clat= clat[1:-1]
    clat0 = clat.split(",")
    f.close()
with open('clon2.txt') as f:
    clon = f.read()
    clon=clon[1:-1]
    clon0 = clon.split(",")
    f.close()

clat,clon=[],[]
for i in range(len(clat0)):
    clat.append(float(clat0[i]))
    clon.append(float(clon0[i]))

## Definitions

In [28]:
start = datetime(2018, 10, 1) #Start date
# Set Time length [days] and timestep [seconds]
length = 25
duration = timedelta(days=length)
dt = 90 #toggle between - or + to pick backwards or forwards
N = len(clat) # number of deploying locations
n = 1 # 1000   # number of particles per location
dmin = list(np.zeros(len(clat)))
#dmin = [0,0,0,0,0,70] #minimum depth
dd = 5 #max depth difference from dmin
x_offset, y_offset, zvals = p_deploy(N,n,dmin,dd)

In [29]:
from parcels import Variable

class MPParticle(JITParticle):        
    ro = Variable('ro', initial = 1380)           
    diameter = Variable('diameter', initial = 1.6e-5)
    length = Variable('length', initial = 60e-5)
    sediment = Variable('sediment', initial = 0)


In [30]:
d=2e-5
l=60e-5
dro=1380-1024
g=9.8
visc=1e-3 
86400*((l/d)**-1.664)*0.079*((l**2)*g*(dro))/(visc)

29.866865098259517

In [31]:
lon = np.zeros([N,n])
lat = np.zeros([N,n])
# Execute run
#clon, clat = [-123.901172,-125.155849,-123.207648,-122.427508,-123.399769,float(outf_lon)], [49.186308,49.975326,49.305448,47.622403,48.399420,float(outf_lat)]  # choose horizontal centre of the particle cloud
for i in range(N):
    lon[i,:]=(clon[i] + x_offset[i,:])
    lat[i,:]=(clat[i] + y_offset[i,:])
z = zvals

In [32]:
#Set start date time and the name of the output file
name = 'uniformH' #name output file
daterange = [start+timedelta(days=i) for i in range(length)]
fn =  name + '_'.join(d.strftime('%Y%m%d')+'_1n' for d in [start, start+duration]) + '.nc'
outfile = os.path.join(paths['out'], fn)
print(outfile)

/home/jvalenti/MOAD/analysis-jose/notebooks/results/uniformH20181001_1n_20181026_1n.nc


## Simulation

In [33]:
#Fill in the list of variables that you want to use as fields
varlist=['U','V','W','R']
filenames,variables,dimensions=filename_set(start,length,varlist,local)
field_set=FieldSet.from_nemo(filenames, variables, dimensions, allow_time_extrapolation=True)

varlist=['US','VS','WL']
filenames,variables,dimensions=filename_set(start,length,varlist,local)

us = Field.from_netcdf(filenames['US'], variables['US'], dimensions,allow_time_extrapolation=True)
vs = Field.from_netcdf(filenames['VS'], variables['VS'], dimensions,allow_time_extrapolation=True)
wl = Field.from_netcdf(filenames['WL'], variables['WL'], dimensions,allow_time_extrapolation=True)
field_set.add_field(us)
field_set.add_field(vs)
field_set.add_field(wl)
field_set.add_vector_field(VectorField("stokes", us, vs, wl))

filenames,variables,dimensions=filename_set(start,length,['Bathy'],local)
Bth = Field.from_netcdf(filenames['Bathy'], variables['Bathy'], dimensions,allow_time_extrapolation=True)
field_set.add_field(Bth)

In [34]:
# #Load Salish output as fields
#field_set = FieldSet.from_nemo(filenames, variables, dimensions, allow_time_extrapolation=True)
pset = ParticleSet.from_list(field_set, MPParticle, lon=lon, lat=lat, depth=z, time=start+timedelta(hours=2))

k_sink = pset.Kernel(Buoyancy)
k_waves = pset.Kernel(Stokes_drift)

pset.execute(AdvectionRK4_3D + k_sink + k_waves,
             runtime=duration, 
             dt=dt,
             output_file=pset.ParticleFile(name=outfile, outputdt=timedelta(hours=1)),
             recovery={ErrorCode.ErrorOutOfBounds: DeleteParticle})

INFO: Compiled ArrayMPParticleAdvectionRK4_3DBuoyancyStokes_drift ==> /tmp/parcels-2894/lib4c7756519e69da792b1cfc01cd0cdc1e_0.so


Particle 5006 lost !! [5400.0, 4.372155107594942, 50.444139681698495, -126.01516994445035]
Particle 5020 lost !! [5400.0, 3.6825826567083, 50.45330268425988, -126.04611057040702]
Particle 5021 lost !! [5400.0, 2.3442939880119704, 50.45961640285889, -126.02683774367233]
Particle 5029 lost !! [5400.0, 3.096377282098228, 50.463155273921174, -126.09947399799537]
Particle 5030 lost !! [5400.0, 4.2812507760495215, 50.4784503899181, -126.06939640376527]
Particle 5037 lost !! [5400.0, 2.2795799178154286, 50.46922062206277, -126.14663841636823]
Particle 5038 lost !! [5400.0, 3.5310708271354727, 50.48411781022563, -126.12163567310428]
Particle 5039 lost !! [5400.0, 2.431488066291329, 50.495919369720646, -126.0881203974508]
Particle 5007 lost !! [15120.0, 7.590312415967556, 50.44070231787224, -125.99993798780618]
Particle 5008 lost !! [15120.0, 2.54413882148082, 50.46573456531847, -125.99999406102366]
Particle 5022 lost !! [15210.0, 6.915553437120396, 50.46883302193846, -125.99977057803702]


INFO: Temporary output files are stored in /home/jvalenti/MOAD/analysis-jose/notebooks/results/out-BPSTGODI.
INFO: You can use "parcels_convert_npydir_to_netcdf /home/jvalenti/MOAD/analysis-jose/notebooks/results/out-BPSTGODI" to convert these to a NetCDF file during the run.
N/A% (0 of 2160000.0) |                  | Elapsed Time: 0:00:00 ETA:  --:--:--

Particle 5009 lost !! [18990.0, 3.1832204362264447, 50.47249630344945, -125.99996367523786]


  0% (14400.0 of 2160000.0) |            | Elapsed Time: 0:00:00 ETA:   0:01:41

...

  5% (111600.0 of 2160000.0) |           | Elapsed Time: 0:01:40 ETA:   0:25:54

Particle 4972 lost !! [118710.0, 9.098354107237233, 50.435900573892084, -125.99992059053378]


 13% (300600.0 of 2160000.0) |#          | Elapsed Time: 0:04:45 ETA:   0:50:53

Particle 4917 lost !! [307710.0, 8.250511230201429, 50.43968941644339, -125.99983386380792]


100% (2160000.0 of 2160000.0) |##########| Elapsed Time: 0:32:46 Time:  0:32:46
/home/jvalenti/conda_envs/parcels/lib/python3.9/site-packages/numpy/lib/arraysetops.py:270: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ar = np.asanyarray(ar)
